In [1]:
import geopandas
import pygeos
import pandas as pd
from time import time

from momepy_utils import get_edge_ratios, get_nodes

In [ ]:
def link(chunk_id):
    s = time()
    cells = geopandas.read_parquet(f"../../urbangrammar_samba/spatial_signatures/morphometrics/cells/cells_{chunk_id}.pq")
    edges = geopandas.read_parquet(f"../../urbangrammar_samba/spatial_signatures/morphometrics/edges/edges_{chunk_id}.pq")
    nodes = geopandas.read_parquet(f"../../urbangrammar_samba/spatial_signatures/morphometrics/nodes/nodes_{chunk_id}.pq")
    
    cells['edgeID'] = get_edge_ratios(cells, edges)
    cells['nodeID'] = get_nodes(cells, nodes, edges, 'nodeID', 'edgeID', 'node_start', 'node_end')
    
    characters = ['sdsSPW', 'sdsSWD', 'sdsSPO', 'sdsLen', 'sssLin', 'ldsMSL']
    l = []
    for d in cells.edgeID:
        l.append((edges.iloc[list(d.keys())][characters].multiply(list(d.values()), axis='rows')).sum(axis=0))
    cells[characters] = pd.DataFrame(l, index=cells.index)
    
    cells = cells.merge(nodes.drop(columns=['geometry']), on='nodeID', how='left')
    cells = cells.rename({'degree': 'mtdDeg', 'meshedness': 'lcdMes', 'proportion_3': 'linP3W', 'proportion_4': 'linP4W',
                     'proportion_0': 'linPDE', 'local_closeness': 'lcnClo'}, axis='columns')
    
    cells['edgeID_keys'] = cells.edgeID.apply(lambda d: list(d.keys()))
    cells['edgeID_values'] = cells.edgeID.apply(lambda d: list(d.values()))
    
    cells.drop(columns='edgeID').to_parquet(f"../../urbangrammar_samba/spatial_signatures/morphometrics/cells/cells_{chunk_id}.pq")
    
    return f"Chunk {chunk_id} processed sucessfully in {time() - s} seconds."

In [ ]:
from dask.distributed import Client, LocalCluster, as_completed

In [ ]:
workers = 14
client = Client(LocalCluster(n_workers=workers, threads_per_worker=1))
client

In [5]:
%%time
inputs = iter(range(103))
futures = [client.submit(link, next(inputs)) for i in range(workers)]
ac = as_completed(futures)
for finished_future in ac:
    # submit new future 
    try:
        new_future = client.submit(link, next(inputs))
        ac.add(new_future)
    except StopIteration:
        pass
    print(finished_future.result())

Chunk 11 processed sucessfully in 566.2533161640167 seconds.
Chunk 3 processed sucessfully in 574.3479053974152 seconds.
Chunk 10 processed sucessfully in 607.8959333896637 seconds.
Chunk 0 processed sucessfully in 614.784334897995 seconds.
Chunk 2 processed sucessfully in 669.9810400009155 seconds.
Chunk 9 processed sucessfully in 680.3808822631836 seconds.
Chunk 1 processed sucessfully in 710.4152896404266 seconds.
Chunk 7 processed sucessfully in 773.7010431289673 seconds.
Chunk 13 processed sucessfully in 777.4415595531464 seconds.
Chunk 8 processed sucessfully in 818.0839283466339 seconds.
Chunk 4 processed sucessfully in 859.4516260623932 seconds.
Chunk 12 processed sucessfully in 1117.8577580451965 seconds.
Chunk 15 processed sucessfully in 569.5318143367767 seconds.
Chunk 17 processed sucessfully in 604.9439563751221 seconds.
Chunk 18 processed sucessfully in 555.3542995452881 seconds.
Chunk 5 processed sucessfully in 1231.6413190364838 seconds.
Chunk 16 processed sucessfully i

In [8]:
client.close()